In [ ]:
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.layers import Conv2DTranspose, UpSampling2D, add
from skimage.transform import resize, rescale
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
from scipy import ndimage, misc
from matplotlib import pyplot
import tensorflow as tf
import numpy as np
np.random.seed(0)
import re
import os
import cv2
import matplotlib.image as mpimg
from matplotlib.pyplot import imsave
import matplotlib.image as mpimg


In [ ]:
import re
from skimage import img_as_ubyte
import imageio

In [ ]:
autoencoder_bilinear = tf.keras.models.load_model('Autencoder_Deconv_0ep107.h5')

In [ ]:
autoencoder_bilinear.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_20 (Conv2D)              (None, None, None, 6 1792        input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_21 (Conv2D)              (None, None, None, 6 36928       conv2d_20[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, None, None, 6 0           conv2d_21[0][0]                  
____________________________________________________________________________________________

In [ ]:
import cv2

In [ ]:
def checkSize(height):
    if (height % 2) == 0:
        #print("{0} is Even".format(height))
        height=int(height/2)
        if (height % 2) == 0:
            #print("{0} is Even".format(height))
            height=int(height/2)
            if (height % 2) == 0:
               # print("{0} is Even".format(height))
                return True
            else:
               # print("{0} is Odd".format(height))
                return False
        else:
           # print("{0} is Odd".format(height))
            return False
    else:
        #print("{0} is Odd".format(height))
        return False
    return False


In [ ]:
def paddingHeight(img, H, W, C):
    
    zimg = np.zeros((H+1, W, C))
    zimg[:H, :W, :C] = img
    
    # Pad the first/last two col and row
    zimg[H:H+1, :W, :] = img[H-1:H , :W, :]    
    return zimg

In [ ]:
def paddingWidth(img, H, W, C):
    
    zimg = np.zeros((H, W+1, C))
    zimg[:H ,:W, :C] = img
    # Pad the first/last two col and row
    zimg[ :H,W:W+1, :] = img[ :H,W-1:W , :]
    
    return zimg

In [ ]:
def deletepadding(image,padding_width,padding_height):
    global Img
    if(padding_width ==0):
        if(padding_height ==0):
            Img=image
            return Img
        else:
            Img=image[:image.shape[0]-padding_height,:image.shape[1],:image.shape[2]]
           
            return Img
    else:
        imageRemoveWidth=image[:image.shape[0] ,:image.shape[1]-padding_width,:image.shape[2]]
        deletepadding(imageRemoveWidth,0,padding_height)
        
    return Img

In [ ]:
def testSize(imagelow,height, width, channels):
    #sr1=[]padding_height
    if( checkSize(height)):
        if( checkSize(width)):
            image= imagelow.astype(np.float32)/255               
            image=[image]
            image=np.array(image)
            global sr1
            print("********************************************************")
            print(str(os.path.splitext(os.path.basename(os.path.normpath(filepath)))[0]))
            t0= time.time()
            sr1 = np.clip(autoencoder_bilinear.predict(image), 0.0, 1.0)
            t1 = time.time() - t0
            print("Time elapsed: ", t1)
            print("********************************************************")

        else:
            image=paddingWidth(imagelow,height, width, channels)
            global padding_width
            padding_width =padding_width+1
            height, width, channels = image.shape
            testSize(image,height, width, channels)
    else:
        image=paddingHeight(imagelow,height, width, channels)
        global padding_height
        padding_height=padding_height+1
        height, width, channels = image.shape
        x=testSize(image,height, width, channels)
    
    return sr1,padding_width,padding_height

In [ ]:
for root, dirnames, filenames in os.walk("./X2X2SR_images/"):
    for filename in filenames:
        if re.search("\.(jpg|jpeg|JPEG|png|bmp|tiff)$", filename):
            
            filepath = os.path.join(root, filename)
            image=cv2.imread(filepath)
            imagelow = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            height, width, channels = imagelow.shape
            padding_width=0
            padding_height=0
            
            
            Imagetest,padding_width,padding_height =testSize(imagelow,height, width, channels)
            
            yimage=deletepadding(Imagetest[0],padding_width*2,padding_height*2)
            
            
            imageio.imwrite("X2X2SR/"+"X2X2Autoencoder"+str(os.path.splitext(os.path.basename(os.path.normpath(filepath)))[0])+".png",img_as_ubyte(yimage))


image in main input  (256, 256, 3)
output with add padding  (1, 512, 512, 3)
edd (512, 512, 3)
image in main input  (144, 144, 3)
output with add padding  (1, 288, 288, 3)
edd (288, 288, 3)
image in main input  (128, 128, 3)
output with add padding  (1, 256, 256, 3)
edd (256, 256, 3)
image in main input  (140, 140, 3)
output with add padding  (1, 288, 288, 3)
edd (280, 280, 3)
image in main input  (172, 114, 3)
output with add padding  (1, 352, 240, 3)
edd (344, 228, 3)
